In [10]:
!pip3 install numpy
!pip3 install os
!pip3 install random
!pip3 install cv2
!pip3 install tensorflow
!pip3 install sklearn
!pip3 install matplotlib

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [9]:
# Import libraries
import numpy as np
import os
import random
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt3

ModuleNotFoundError: No module named 'numpy'

In [ ]:
#Dataset link: https://www.kaggle.com/datasets/ninadaithal/imagesoasis

#Replace this with the path
dataset_folder = 'OASIS Data'

In [ ]:

classes = ['Non Demented', 'Mild Dementia', 'Moderate Dementia', 'Very mild Dementia']

image_paths = []
labels = []

# Parameters
num_files = 488  # Number of files to select randomly from each category
dementia_classes = ['Mild Dementia', 'Moderate Dementia', 'Very mild Dementia']

# Iterate through Non-Demented class
non_demented_path = os.path.join(dataset_folder, 'Non Demented')
non_demented_files = os.listdir(non_demented_path)
print(f"Non Demented category contains {len(non_demented_files)} files.")

# Randomly select files from Non Demented class
non_demented_files = random.sample(non_demented_files, min(num_files, len(non_demented_files)))
for image_filename in non_demented_files:
    image_path = os.path.join(non_demented_path, image_filename)
    if os.path.isfile(image_path):
        image_paths.append(image_path)
        labels.append(0)  # Label for Non Demented

# Iterate through Dementia classes (Mild, Moderate, Very Mild)
dementia_images = []
dementia_labels = []

for category in dementia_classes:
    category_path = os.path.join(dataset_folder, category)
    category_files = os.listdir(category_path)
    print(f"Category '{category}' contains {len(category_files)} files.")
    
    # Randomly select files from each Dementia class
    selected_files = random.sample(category_files, min(num_files // len(dementia_classes), len(category_files)))
    print(f"Randomly selected {len(selected_files)} files from '{category}'.")
    
    for image_filename in selected_files:
        image_path = os.path.join(category_path, image_filename)
        if os.path.isfile(image_path):
            dementia_images.append(image_path)
            dementia_labels.append(1)  # Label for Dementia (combined category)

In [ ]:
 Combine Non Demented and Dementia data
image_paths.extend(dementia_images)
labels.extend(dementia_labels)

# Convert labels list to a numpy array
labels = np.array(labels)

# Load images function
def load_images(paths, img_size=(128, 128)):
    images = []
    for path in paths:
        # Read image (grayscale for simplicity)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)  # Resize to fixed size
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load the images based on selected paths
X = load_images(image_paths)

In [ ]:
subjects = [image_path.split(os.path.sep)[-1].split('OAS1_')[1].split('_')[0] for image_path in image_paths]  # Extract subject ID

# Perform train/test split by subject to prevent data leakage
unique_subjects = list(set(subjects))
train_subjects, val_subjects = train_test_split(unique_subjects, test_size=0.3, random_state=42)

print(len(unique_subjects))


# Split images based on subject IDs
train_indices = [i for i, subject in enumerate(subjects) if subject in train_subjects]
val_indices = [i for i, subject in enumerate(subjects) if subject in val_subjects]

X_train = X[train_indices]
y_train = labels[train_indices]
X_val = X[val_indices]
y_val = labels[val_indices]

# Reshape the data to include the channel dimension (grayscale)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]


In [ ]:
# Define a simple CNN model for binary classification
def create_model(input_shape=(128, 128, 1)):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification (output: 0 or 1)
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the model
model = create_model()

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=32)

# You can plot the training history to evaluate the model's performance
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()